<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Data Wrangling Lab**


Estimated time needed: **45** minutes


In this lab, you will perform data wrangling tasks to prepare raw data for analysis. Data wrangling involves cleaning, transforming, and organizing data into a structured format suitable for analysis. This lab focuses on tasks like identifying inconsistencies, encoding categorical variables, and feature transformation.


## Objectives


After completing this lab, you will be able to:


- Identify and remove inconsistent data entries.

- Encode categorical variables for analysis.

- Handle missing values using multiple imputation strategies.

- Apply feature scaling and transformation techniques.


#### Intsall the required libraries


In [ ]:
!pip install pandas
!pip install matplotlib

## Tasks


#### Step 1: Import the necessary module.


### 1. Load the Dataset


<h5>1.1 Import necessary libraries and load the dataset.</h5>


Ensure the dataset is loaded correctly by displaying the first few rows.


In [1]:
# Import necessary libraries
import pandas as pd

# Load the Stack Overflow survey data
dataset_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/n01PQ9pSmiRX6520flujwQ/survey-data.csv"
df = pd.read_csv(dataset_url)

# Display the first few rows
print(df.head())


   ResponseId                      MainBranch                 Age  \
0           1  I am a developer by profession  Under 18 years old   
1           2  I am a developer by profession     35-44 years old   
2           3  I am a developer by profession     45-54 years old   
3           4           I am learning to code     18-24 years old   
4           5  I am a developer by profession     18-24 years old   

            Employment RemoteWork   Check  \
0  Employed, full-time     Remote  Apples   
1  Employed, full-time     Remote  Apples   
2  Employed, full-time     Remote  Apples   
3   Student, full-time        NaN  Apples   
4   Student, full-time        NaN  Apples   

                                    CodingActivities  \
0                                              Hobby   
1  Hobby;Contribute to open-source projects;Other...   
2  Hobby;Contribute to open-source projects;Other...   
3                                                NaN   
4                                 

#### 2. Explore the Dataset


<h5>2.1 Summarize the dataset by displaying the column data types, counts, and missing values.</h5>


In [10]:
#Display data types and non-null counts
print("=== Dataset Info ===")
df.info()

# Display missing values for each column
print("\n=== Missing Values ===")
missing_values = df.isnull().sum()
missing_percent = (df.isnull().mean() * 100).round(2)
missing_summary = pd.DataFrame({
    'Missing Values': missing_values,
    'Missing (%)': missing_percent,
    'Data Type': df.dtypes
})
print(missing_summary[missing_summary['Missing Values'] > 0])

=== Dataset Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65437 entries, 0 to 65436
Columns: 114 entries, ResponseId to JobSat
dtypes: float64(13), int64(1), object(100)
memory usage: 56.9+ MB

=== Missing Values ===
                     Missing Values  Missing (%) Data Type
RemoteWork                    10631        16.25    object
CodingActivities              10971        16.77    object
EdLevel                        4653         7.11    object
LearnCode                      4949         7.56    object
LearnCodeOnline               16200        24.76    object
...                             ...          ...       ...
JobSatPoints_11               35992        55.00   float64
SurveyLength                   9255        14.14    object
SurveyEase                     9199        14.06    object
ConvertedCompYearly           42002        64.19   float64
JobSat                        36311        55.49   float64

[109 rows x 3 columns]


<h5>2.2 Generate basic statistics for numerical columns.</h5>


In [11]:
# Write your code here
df.describe()

,ResponseId,CompTotal,WorkExp,JobSatPoints_1,JobSatPoints_4,JobSatPoints_5,JobSatPoints_6,JobSatPoints_7,JobSatPoints_8,JobSatPoints_9,JobSatPoints_10,JobSatPoints_11,ConvertedCompYearly,JobSat
count,65437.000000,3.374000e+04,29658.000000,29324.000000,29393.000000,29411.000000,29450.000000,29448.00000,29456.000000,29456.000000,29450.000000,29445.000000,2.343500e+04,29126.000000
mean,32719.000000,2.963841e+145,11.466957,18.581094,7.522140,10.060857,24.343232,22.96522,20.278165,16.169432,10.955713,9.953948,8.615529e+04,6.935041
std,18890.179119,5.444117e+147,9.168709,25.966221,18.422661,21.833836,27.089360,27.01774,26.108110,24.845032,22.906263,21.775652,1.867570e+05,2.088259
min,1.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000e+00,0.000000
25%,16360.000000,6.000000e+04,4.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,3.271200e+04,6.000000
50%,32719.000000,1.100000e+05,9.000000,10.000000,0.000000,0.000000,20.000000,15.00000,10.000000,5.000000,0.000000,0.000000,6.500000e+04,7.000000
75%,49078.000000,2.500000e+05,16.000000,22.000000,5.000000,10.000000,30.000000,30.00000,25.000000,20.000000,10.000000,10.000000,1.079715e+05,8.000000
max,65437.000000,1.000000e+150,50.000000,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000,1.625660e+07,10.000000


### 3. Identifying and Removing Inconsistencies


<h5>3.1 Identify inconsistent or irrelevant entries in specific columns (e.g., Country).</h5>


In [16]:
# Write your code here
df['Country'].unique()
df['Country'].value_counts(dropna=False)
df['Country'].isnull().sum() 
#(df['Country'].str.strip() == '').sum()

6507

<h5>3.2 Standardize entries in columns like Country or EdLevel by mapping inconsistent values to a consistent format.</h5>


In [31]:
df['Country'].value_counts(dropna=False)

Country
USA                 11095
NaN                  6507
GERMANY              4947
INDIA                4231
UNITED KINGDOM       3224
                    ...  
NIGER                   1
GUINEA                  1
DOMINICA                1
PAPUA NEW GUINEA        1
SOLOMON ISLANDS         1
Name: count, Length: 186, dtype: int64

In [26]:
## Write your code here
import numpy as np
df['Country'] = df['Country'].str.upper().str.strip()
country_mapping = {
    'UNTED STATES': 'USA',
    'U.S.A': 'USA',
    'UNITED STATES OF AMERICA': 'USA',
    'US': 'USA',
    'UK': 'UNITED KINGDOM',
    'UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN IRELAND': 'UNITED KINGDOM',
    'ENGLAND': 'UNITED KINGDOM',
    'SCOTLAND': 'UNITED KINGDOM',
    'N/A': np.nan,
    'NONE': np.nan,
    '': np.nan
}
df['Country'] = df['Country'].replace(country_mapping)
df['Country'] 

0                   USA
1        UNITED KINGDOM
2        UNITED KINGDOM
3                CANADA
4                NORWAY
              ...      
65432               NaN
65433               NaN
65434               NaN
65435           GERMANY
65436               NaN
Name: Country, Length: 65437, dtype: object

### 4. Encoding Categorical Variables


<h5>4.1 Encode the Employment column using one-hot encoding.</h5>


In [37]:
## Write your code here
df['Employment'].unique()
df['Employment'].isnull().sum()
df['Employment'] = df['Employment'].fillna('Unknown')

employment_encoded = pd.get_dummies(df['Employment'], prefix='Employment')

# Combine the encoded columns with the original dataframe (optional)
df_encoded = pd.concat([df, employment_encoded], axis=1)

# Optionally, drop the original 'Employment' column if no longer needed
df_encoded = df_encoded.drop('Employment', axis=1)


### 5. Handling Missing Values


<h5>5.1 Identify columns with the highest number of missing values.</h5>


In [40]:
## Write your code here
df.isnull().sum().sort_values(ascending=False)

AINextMuch less integrated    64289
AINextLess integrated         63082
AINextNo change               52939
AINextMuch more integrated    51999
EmbeddedAdmired               48704
                              ...  
MainBranch                        0
Check                             0
Employment                        0
Age                               0
ResponseId                        0
Length: 114, dtype: int64

<h5>5.2 Impute missing values in numerical columns (e.g., `ConvertedCompYearly`) with the mean or median.</h5>


In [41]:
## Write your code here
median_value = df['ConvertedCompYearly'].median()
df['ConvertedCompYearly'] = df['ConvertedCompYearly'].fillna(median_value)

<h5>5.3 Impute missing values in categorical columns (e.g., `RemoteWork`) with the most frequent value.</h5>


In [42]:
## Write your code here
fre_value = df['RemoteWork'].mode()[0]
df['RemoteWork'] = df['RemoteWork'].fillna(fre_value)

In [44]:
# Impute all numeric columns with their median
df.fillna(df.median(numeric_only=True), inplace=True)

In [45]:
df.isnull().sum().sort_values(ascending=False)

AINextMuch less integrated    64289
AINextLess integrated         63082
AINextNo change               52939
AINextMuch more integrated    51999
EmbeddedAdmired               48704
                              ...  
Check                             0
RemoteWork                        0
Employment                        0
Age                               0
JobSat                            0
Length: 114, dtype: int64

### 6. Feature Scaling and Transformation


<h5>6.1 Apply Min-Max Scaling to normalize the `ConvertedCompYearly` column.</h5>


In [49]:
## Write your code here
min_year=df['ConvertedCompYearly'].min()
max_year=df['ConvertedCompYearly'].max()
df['ConvertedCompYearly_MinMax']=(df['ConvertedCompYearly']-min_year)/(max_year-min_year)
df[['ConvertedCompYearly','ConvertedCompYearly_MinMax']]

,ConvertedCompYearly,ConvertedCompYearly_MinMax
0,65000.0,0.003998
1,65000.0,0.003998
2,65000.0,0.003998
3,65000.0,0.003998
4,65000.0,0.003998
...,...,...
65432,65000.0,0.003998
65433,65000.0,0.003998
65434,65000.0,0.003998
65435,65000.0,0.003998


<h5>6.2 Log-transform the ConvertedCompYearly column to reduce skewness.</h5>


In [50]:
## Write your code here
df['LogSalary'] = np.log1p(df['ConvertedCompYearly'])

### 7. Feature Engineering


<h5>7.1 Create a new column `ExperienceLevel` based on the `YearsCodePro` column:</h5>


In [52]:
## Write your code here
import pandas as pd

# Example: cleaning and converting YearsCodePro
def clean_years_code(value):
    if pd.isnull(value):
        return None
    elif value == 'Less than 1 year':
        return 0.5
    elif value == 'More than 50 years':
        return 51
    else:
        try:
            return float(value)
        except:
            return None

# Apply the cleaning function
df['YearsCodePro_clean'] = df['YearsCodePro'].apply(clean_years_code)

# Define experience level mapping
def experience_level(years):
    if pd.isnull(years):
        return 'Unknown'
    elif years < 3:
        return 'Beginner'
    elif 3 <= years < 8:
        return 'Intermediate'
    else:
        return 'Expert'

# Create the new column
df['ExperienceLevel'] = df['YearsCodePro_clean'].apply(experience_level)
df['ExperienceLevel'] 

0             Unknown
1              Expert
2              Expert
3             Unknown
4             Unknown
             ...     
65432    Intermediate
65433         Unknown
65434    Intermediate
65435        Beginner
65436         Unknown
Name: ExperienceLevel, Length: 65437, dtype: object

### Summary


In this lab, you:

- Explored the dataset to identify inconsistencies and missing values.

- Encoded categorical variables for analysis.

- Handled missing values using imputation techniques.

- Normalized and transformed numerical data to prepare it for analysis.

- Engineered a new feature to enhance data interpretation.


Copyright © IBM Corporation. All rights reserved.
